In [1]:
# Ignore warnings

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Check Python Environments

import sys

print( 'Python Version:', sys.version.split()[0] )

Python Version: 3.7.6


In [3]:
# Check GPU available on container
!nvidia-smi -L

GPU 0: GeForce GTX 1070 Ti (UUID: GPU-48b7cc91-97d8-36be-f438-35c05dead2ec)
GPU 1: GeForce GTX 1070 Ti (UUID: GPU-4ebfd3a3-6fac-600e-aabf-9230eba08522)


In [4]:
# select CUDA device with PCI BUS ID
# note: default CUDA will sort by speed (GPU 0 is fastest GPU)

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

print( 'CUDA Version:', os.environ['CUDA_VERSION'] )

CUDA Version: 10.1.243


In [5]:
# Check Tensorflow Environments

import tensorflow as tf

print( 'Tensorflow Version:', tf.__version__)
print("GPU Available::", tf.config.list_physical_devices('GPU'))

# note Tensorflow see GPU from PCI BUS ID

Tensorflow Version: 2.1.0
GPU Available:: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
# Load mnist
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print( 'x_train: {}'.format(str(x_train.shape)), 'y_train: {}'.format(str(y_train.shape)) )
print( 'x_test: {}'.format(str(x_test.shape)), 'y_test: {}'.format(str(y_test.shape)) )

x_train: (60000, 28, 28) y_train: (60000,)
x_test: (10000, 28, 28) y_test: (10000,)


In [7]:
# Prepare data
from tensorflow.keras.utils import to_categorical
import numpy as np

x_train = x_train.reshape( x_train.shape + (1,) ).astype(np.float32) / 255.
x_test = x_test.reshape( x_test.shape + (1,) ).astype(np.float32) / 255.

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

print( 'x_train: {}'.format(str(x_train.shape)), 'y_train: {}'.format(str(y_train.shape)) )
print( 'x_test: {}'.format(str(x_test.shape)), 'y_test: {}'.format(str(y_test.shape)) )

x_train: (60000, 28, 28, 1) y_train: (60000, 10)
x_test: (10000, 28, 28, 1) y_test: (10000, 10)


In [8]:
# Define model with Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.activations import relu, softmax

model = Sequential(name="my_model")
model.add(Conv2D(32, kernel_size=(3,3), activation=relu, input_shape=x_train.shape[1:] ))
model.add(Conv2D(64, (3, 3), activation=relu))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation=relu))
model.add(Dropout(0.5))
model.add(Dense(y_train.shape[1], activation=softmax))

model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=[categorical_accuracy])

model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0  

In [9]:
# Train model

history = model.fit(
    x_train, y_train, 
    batch_size=128, 
    epochs=5, 
    verbose=1, 
    validation_data=(x_test, y_test)
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 6s 98us/sample - loss: 0.2616 - categorical_accuracy: 0.9215 - val_loss: 0.0546 - val_categorical_accuracy: 0.9827
Epoch 2/5
60000/60000 [==============================] - 5s 85us/sample - loss: 0.0908 - categorical_accuracy: 0.9739 - val_loss: 0.0396 - val_categorical_accuracy: 0.9867
Epoch 3/5
60000/60000 [==============================] - 5s 89us/sample - loss: 0.0656 - categorical_accuracy: 0.9800 - val_loss: 0.0377 - val_categorical_accuracy: 0.9881
Epoch 4/5
60000/60000 [==============================] - 5s 89us/sample - loss: 0.0559 - categorical_accuracy: 0.9826 - val_loss: 0.0317 - val_categorical_accuracy: 0.9904
Epoch 5/5
60000/60000 [==============================] - 5s 88us/sample - loss: 0.0460 - categorical_accuracy: 0.9859 - val_loss: 0.0285 - val_categorical_accuracy: 0.9911


In [10]:
# Test model

loss, acc = model.evaluate(x_test, y_test, verbose=1)

print('test loss: {}'.format(loss))
print('test acc: {}'.format(acc))

10000/10000 [==============================] - 1s 85us/sample - loss: 0.0285 - categorical_accuracy: 0.9911
test loss: 0.02846179952417151
test acc: 0.991100013256073
